In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, auc, average_precision_score
#import scikitplot as skplt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import keras.backend as K
from NGF.preprocessing import tensorise_smiles
from custom_layers.model_creator import encode_smiles, stage_creator
from keras.layers import Dense, Dropout, Input, Lambda, concatenate,Flatten
from keras.models import Model, load_model
import dill
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
import xgboost as xgb
from math import ceil
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
def pairwise_distance(feature, squared):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums
def triplet_loss_adapted_from_tf(y_true, y_pred):
    del y_true
    margin = 0.5
    labels = y_pred[:, :1]
    labels = tf.cast(labels, dtype='int32')
    embeddings = y_pred[:, 1:]
    
    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=False)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

In [3]:
target_1 = 'p38'
base_path_1 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_1 = base_path_1+f'/data/{target_1}/data.csv'
df_p38=pd.read_csv(data_fpath_1).set_index('biolab_index')

with open(base_path_1+f'/data/{target_1}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_p38 = dill.load(in_f)

with open(base_path_1+f'/data/{target_1}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_p38 = dill.load(in_f)
    
target_2 = 'akt1'
base_path_2 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_2 = base_path_2+f'/data/{target_2}/data.csv'
df_akt1 = pd.read_csv(data_fpath_2).set_index('biolab_index')

with open(base_path_2+f'/data/{target_2}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_akt1 = dill.load(in_f)
with open(base_path_2+f'/data/{target_2}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_akt1 = dill.load(in_f)
    
target_3 = 'pi3k'
base_path_3 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_3 = base_path_3+f'/data/{target_3}/data.csv'
df_pi3k = pd.read_csv(data_fpath_3).set_index('biolab_index')

with open(base_path_3+f'/data/{target_3}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_pi3k = dill.load(in_f)
with open(base_path_3+f'/data/{target_3}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_pi3k = dill.load(in_f)

In [4]:
train_p38 = df_p38.loc[train_test_folds_p38[0]]            
val_p38 = df_p38.loc[train_test_folds_p38[1]]
                   

train_akt1 = df_akt1.loc[train_test_folds_akt1[0]]           
val_akt1 = df_akt1.loc[train_test_folds_akt1[1]]
                   

train_pi3k = df_pi3k.loc[train_test_folds_pi3k[0]]        
val_pi3k = df_pi3k.loc[train_test_folds_pi3k[1]]

In [5]:
#Random splits with sklearn (on our test set)
df_p38 = df_p38.reset_index(drop=True)
X_train_p38, X_val_p38, Y_train_p38, Y_val_p38 = train_test_split(df_p38.rdkit,
                                                                  df_p38.Binary,
                                                                  test_size = 0.15,
                                                                  train_size = 0.85,
                                                                  shuffle = True)
random_train_p38 = pd.DataFrame(X_train_p38)
random_train_p38['Binary'] = Y_train_p38
random_val_p38 = pd.DataFrame(X_val_p38)
random_val_p38['Binary'] = Y_val_p38
del X_train_p38,X_val_p38,Y_train_p38,Y_val_p38


df_akt1 = df_akt1.reset_index(drop=True)
X_train_akt1, X_val_akt1, Y_train_akt1, Y_val_akt1 = train_test_split(df_akt1.rdkit,
                                                                     df_akt1.Binary,
                                                                     test_size = 0.15,
                                                                     train_size = 0.85,
                                                                     shuffle = True)
random_train_akt1 = pd.DataFrame(X_train_akt1)
random_train_akt1['Binary'] = Y_train_akt1
random_val_akt1 = pd.DataFrame(X_val_akt1)
random_val_akt1['Binary'] = Y_val_akt1
del X_train_akt1,X_val_akt1,Y_train_akt1,Y_val_akt1


df_pi3k = df_pi3k.reset_index(drop=True)
X_train_pi3k, X_val_pi3k, Y_train_pi3k, Y_val_pi3k = train_test_split(df_pi3k.rdkit,
                                                                      df_pi3k.Binary,
                                                                      test_size = 0.15,
                                                                      train_size = 0.85,
                                                                      shuffle = True)
random_train_pi3k = pd.DataFrame(X_train_pi3k)
random_train_pi3k['Binary'] = Y_train_pi3k
random_val_pi3k = pd.DataFrame(X_val_pi3k)
random_val_pi3k['Binary'] = Y_val_pi3k
del X_train_pi3k,X_val_pi3k,Y_train_pi3k,Y_val_pi3k


In [6]:
def csv_generator_mining(bs,df_p1,df_p2,df_p3,
                         Y_dummy_p1,Y_dummy_p2,Y_dummy_p3,
                         smiles_list_p1,smiles_list_p2,smiles_list_p3,
                         Y_p1,Y_p2,Y_p3,
                         atoms_p1,bonds_p1,edges_p1,
                         atoms_p2,bonds_p2,edges_p2,
                         atoms_p3,bonds_p3,edges_p3,mode = 'train',aug = None):
    
    counter_1=int(0)
    counter_2=int(0)
    counter_3=int(0)
    #Keep looping indefinetely
    df_p1 = df_p1.reset_index(drop=True)
    df_p2 = df_p2.reset_index(drop=True)
    df_p3 = df_p3.reset_index(drop=True)
    Y_p1 = np.array(list(Y_p1))
    Y_p2 = np.array(list(Y_p2))
    Y_p3 = np.array(list(Y_p3))
    while True:
        
        #Initialize batches of inputs and outputs
        ind1 = []
        ind2 = []
        ind3 = []
        
        d_p1=[]
        d_p2=[]
        d_p3=[]
        
        #Keep looping until we reach batch size
        while len(ind3)<=bs: #doesn't matter if it is smi1 or smi2 since they have the same len
            if counter_3==len(df_p3):
                counter_3=int(0)
                df_p3 = df_p3.sample(frac=1).reset_index(drop=True)
            
            smi_p3 = df_p3['rdkit'][counter_3]
            ind3.append(smiles_list_p3.index(smi_p3))
            d_p3.append(Y_dummy_p3[counter_3])
            counter_3+=1
            while len(ind1)<=bs:
                if counter_1==len(df_p1):
                    counter_1=int(0)
                    df_p1 = df_p1.sample(frac=1).reset_index(drop=True)
                
                smi_p1 = df_p1['rdkit'][counter_1]
                ind1.append(smiles_list_p1.index(smi_p1))
                d_p1.append(Y_dummy_p1[counter_1])
                counter_1+=1
                
                
                while len(ind2)<=bs:
            # check to see if you reached the end of the frame
                    if counter_2==len(df_p2):
                        counter_2=int(0)
                        df_p2 = df_p2.sample(frac=1).reset_index(drop=True)

                    smi_p2 = df_p2['rdkit'][counter_2]
                    ind2.append(smiles_list_p2.index(smi_p2))
                    d_p2.append(Y_dummy_p2[counter_2])
                    counter_2+=1
                # if we are evaluating we should now break from our 
                # loop to ensure we don't continue to fill up the batch from samples at the beginning of the file
                
                   
            
            

            #smi_p1 = df_p1['rdkit'][counter]
            #smi_p2 = df_p2['rdkit'][counter]
            #smi_p3 = df_p3['rdkit'][counter]
            #ind1.append(smiles_list_p1.index(smi_p1))
            #ind2.append(smiles_list_p2.index(smi_p2))
            #ind3.append(smiles_list_p3.index(smi_p3))
            #d_p1.append(Y_dummy_p1[counter])
            #d_p2.append(Y_dummy_p2[counter])
            #d_p3.append(Y_dummy_p3[counter])
            #counter+=1
            
        atoms_target_1 = np.array(atoms_p1[ind1],dtype = 'float32')
        bonds_target_1 = np.array(bonds_p1[ind1],dtype = 'float32')
        edges_target_1 = np.array(edges_p1[ind1],dtype = 'int32')
        labels_target_1 = Y_p1[ind1]
        
        atoms_target_2 = np.array(atoms_p2[ind2],dtype = 'float32')
        bonds_target_2 = np.array(bonds_p2[ind2],dtype = 'float32')
        edges_target_2 = np.array(edges_p2[ind2],dtype = 'int32')
        labels_target_2 = Y_p2[ind2]
        
        atoms_target_3 = np.array(atoms_p3[ind3],dtype = 'float32')
        bonds_target_3 = np.array(bonds_p3[ind3],dtype = 'float32')
        edges_target_3 = np.array(edges_p3[ind3],dtype = 'int32')
        labels_target_3 = Y_p3[ind3]
        
        yield ({'atom_inputs_1':atoms_target_1,
                'bond_inputs_1':bonds_target_1,
                'edge_inputs_1':edges_target_1,
                'labels_inputs_1':labels_target_1,
                'atom_inputs_2':atoms_target_2,
                'bond_inputs_2':bonds_target_2,
                'edge_inputs_2':edges_target_2,
                'labels_inputs_2':labels_target_2,
                'atom_inputs_3':atoms_target_3,
                'bond_inputs_3':bonds_target_3,
                'edge_inputs_3':edges_target_3,
                'labels_inputs_3':labels_target_3},{'Protein_1':np.array(d_p1,dtype = 'float32'),
                                                    'Protein_2':np.array(d_p2,dtype = 'float32'),
                                                    'Protein_3':np.array(d_p3,dtype = 'float32')})

In [7]:
def build_encoder(encoder_params):
        model_enc_1 = stage_creator(encoder_params, 1, conv=True)[0]
        model_enc_2 = stage_creator(encoder_params, 2, conv=True)[0]
        model_enc_3 = stage_creator(encoder_params, 3, conv=True)[0]

        model_enc_fp_1 = stage_creator(encoder_params, 1, conv=False)[1]
        model_enc_fp_2 = stage_creator(encoder_params, 2, conv=False)[1]
        model_enc_fp_3 = stage_creator(encoder_params, 3, conv=False)[1]

        atoms, bonds, edges = encode_smiles(encoder_params["max_atoms"],
                                            encoder_params["num_atom_features"],
                                            encoder_params["max_degree"],
                                            encoder_params["num_bond_features"])

        graph_conv_1 = model_enc_1([atoms, bonds, edges])
        graph_conv_2 = model_enc_2([graph_conv_1, bonds, edges])
        graph_conv_3 = model_enc_3([graph_conv_2, bonds, edges])

        fingerprint_1 = model_enc_fp_1([graph_conv_1, bonds, edges])
        fingerprint_1 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_1)

        fingerprint_2 = model_enc_fp_2([graph_conv_2, bonds, edges])
        fingerprint_2 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_2)

        fingerprint_3 = model_enc_fp_3([graph_conv_3, bonds, edges])
        fingerprint_3 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_3)

        final_fingerprint = keras.layers.add([fingerprint_1, fingerprint_2, fingerprint_3])

        return Model([atoms, bonds, edges], [final_fingerprint])

In [8]:
def build_model(model_params, encoder, verbose=False):
        atoms_1 = Input(name='atom_inputs_1',
                      shape=(model_params['max_atoms'], model_params['num_atom_features']), dtype='float32')
        bonds_1 = Input(name='bond_inputs_1', shape=(
            model_params['max_atoms'], model_params['max_degree'], model_params['num_bond_features']),
                      dtype='float32')
        edges_1 = Input(name='edge_inputs_1', shape=(model_params['max_atoms'], model_params['max_degree']),
                      dtype='int32')
        encode_drug_1 = encoder([atoms_1, bonds_1, edges_1])
        
        atoms_2 = Input(name='atom_inputs_2',
                      shape=(model_params['max_atoms'], model_params['num_atom_features']), dtype='float32')
        bonds_2 = Input(name='bond_inputs_2', shape=(
            model_params['max_atoms'], model_params['max_degree'], model_params['num_bond_features']),
                      dtype='float32')
        edges_2 = Input(name='edge_inputs_2', shape=(model_params['max_atoms'], model_params['max_degree']),
                      dtype='int32')
        encode_drug_2 = encoder([atoms_2, bonds_2, edges_2])
        
        atoms_3 = Input(name='atom_inputs_3',
                      shape=(model_params['max_atoms'], model_params['num_atom_features']), dtype='float32')
        bonds_3 = Input(name='bond_inputs_3', shape=(
            model_params['max_atoms'], model_params['max_degree'], model_params['num_bond_features']),
                      dtype='float32')
        edges_3 = Input(name='edge_inputs_3', shape=(model_params['max_atoms'], model_params['max_degree']),
                      dtype='int32')
        encode_drug_3 = encoder([atoms_3, bonds_3, edges_3])
        
        conc = keras.layers.Concatenate(axis = -1)([encode_drug_1,encode_drug_2,encode_drug_3])
        # Fully connected
        FC1 = Dense(model_params["dense_size"][0], activation='relu',kernel_initializer='random_normal')(conc)
        FC2 = Dropout(model_params["dropout_rate"][0])(FC1)
        FC2 = Dense(model_params["dense_size"][1], activation='relu',kernel_initializer='random_normal')(FC2)
        FC2 = Dropout(model_params["dropout_rate"][1])(FC2)
        FC2 = Dense(model_params["dense_size"][2], activation = None,kernel_initializer='random_normal')(FC2)
        
        
        embeddings_conc = Lambda(lambda x: K.l2_normalize(x,axis=1),name = 'Embeddings_Concatenated')(FC2)
        embeddings_1 = crop(1,0,int(model_params['dense_size'][2]/3),name = 'Target_1')(embeddings_conc) #0 to 255(256)
        embeddings_2 = crop(1,int(model_params['dense_size'][2]/3),2*int(model_params['dense_size'][2]/3),name = 'Target_2')(embeddings_conc) #256 to 511(256)
        embeddings_3 = crop(1,2*int(model_params['dense_size'][2]/3),3*int(model_params['dense_size'][2]/3),name = 'Target_3')(embeddings_conc)
        
        gcn_model = Model(inputs=[atoms_1, bonds_1, edges_1,
                                  atoms_2, bonds_2, edges_2,
                                  atoms_3, bonds_3, edges_3], 
                                  outputs = [embeddings_1,
                                             embeddings_2,
                                             embeddings_3]
                         )


        if verbose:
            #print('encoder')
            #encoder.summary()
            print('GCN_model')
            gcn_model.summary()
        
            
        return gcn_model

In [9]:
def build_mining(model_params,gcn_model):
        #First Target
        atoms_1 = Input(name='atom_inputs_1',shape=(model_params['max_atoms'], 
                                                    model_params['num_atom_features']), 
                                                    dtype='float32')
        bonds_1 = Input(name='bond_inputs_1', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree'], 
                                                     model_params['num_bond_features']),
                                                     dtype='float32')
        edges_1 = Input(name='edge_inputs_1', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree']),
                                                     dtype='int32')
        
        labels_1 = Input(name = 'labels_inputs_1',shape = (1,),dtype = 'float32')
        
        #Second Target
        atoms_2 = Input(name='atom_inputs_2',shape=(model_params['max_atoms'], 
                                                    model_params['num_atom_features']), 
                                                    dtype='float32')
        bonds_2 = Input(name='bond_inputs_2', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree'], 
                                                     model_params['num_bond_features']),
                                                     dtype='float32')
        edges_2 = Input(name='edge_inputs_2', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree']),
                                                     dtype='int32')
        
        labels_2 = Input(name = 'labels_inputs_2',shape = (1,),dtype = 'float32')
        
        #Third Target
        atoms_3 = Input(name='atom_inputs_3',shape=(model_params['max_atoms'], 
                                                    model_params['num_atom_features']), 
                                                    dtype='float32')
        bonds_3 = Input(name='bond_inputs_3', shape=(model_params['max_atoms'], 
                                                     model_params['max_degree'], 
                                                     model_params['num_bond_features']),
                                                     dtype='float32')
        edges_3 = Input(name='edge_inputs_3', shape=(model_params['max_atoms'], model_params['max_degree']),
                      dtype='int32')
        
        labels_3 = Input(name = 'labels_inputs_3',shape = (1,),dtype = 'float32')
        encoded_1,encoded_2,encoded_3 = gcn_model([atoms_1,bonds_1,edges_1,
                                                   atoms_2,bonds_2,edges_2,
                                                   atoms_3,bonds_3,edges_3])
        
        labels_plus_embeddings_1 = concatenate([labels_1, encoded_1],name = 'Protein_1')
        labels_plus_embeddings_2 = concatenate([labels_2, encoded_2],name = 'Protein_2')
        labels_plus_embeddings_3 = concatenate([labels_3, encoded_3],name = 'Protein_3')
        
        mining_net = Model(inputs = [atoms_1,bonds_1,edges_1,labels_1,
                                     atoms_2,bonds_2,edges_2,labels_2,
                                     atoms_3,bonds_3,edges_3,labels_3],
                           outputs = [labels_plus_embeddings_1,
                                      labels_plus_embeddings_2,
                                      labels_plus_embeddings_3])
        adam = keras.optimizers.Adam(lr = model_params["lr"], 
                                     beta_1=0.9, 
                                     beta_2=0.999, 
                                     decay=0.0, 
                                     amsgrad=False)
    
    
        mining_net.compile(optimizer=adam , loss = {'Protein_1' : triplet_loss_adapted_from_tf,
                                                    'Protein_2' : triplet_loss_adapted_from_tf,
                                                    'Protein_3' : triplet_loss_adapted_from_tf})
        mining_net.summary()
        return mining_net

In [10]:
def dataframe_to_gcn_input(input_data):
        x_atoms_cold, x_bonds_cold, x_edges_cold = tensorise_smiles(input_data['rdkit'],
                                                                    max_degree=model_params['max_degree'],
                                                                    max_atoms=model_params['max_atoms'])
        return [x_atoms_cold, x_bonds_cold, x_edges_cold]

In [11]:
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
model_params = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(96), int(104), int(120)],
        "fp_length" : [int(160), int(160), int(160)],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(3*160), int(3*160), int(3*96)], 
        'dropout_rate' : [0.27225175676555935, 0.27225175676555935],
        'lr' : 0.008110012706176706,
        'batch_size' : int(256),
        'n_epochs' : int(20),
        'margin' : 1
        }
xgb_hyper = {
        "colsample_bylevel" : 0.4371082812232264,
        "colsample_bytree" : 0.4179415558635843,
        "gamma" : 0.919836526180396,
        "eta" : 0.41409388868400826,
        "max_delta_step" : int(2),
        "max_depth" : int(7),
        "min_child_weight" : int(20),
        "alpha" : 0.15030685758880047,
        "lambda" : 12.306130216692438,
        "subsample" : 0.6038298323514097,
        "max_bin" : int(48.0),
        "eval_metric":'auc',
        "objective":'binary:logistic',
        "booster":'gbtree'
}

In [12]:
def crop(dimension, start, end,name):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
    def func(x):
        if dimension == 0:
            return x[start: end]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func,name = name)

In [13]:
training_p38 = [train_p38,random_train_p38]
validation_p38 = [val_p38,random_val_p38]

training_akt1 = [train_akt1,random_train_akt1]
validation_akt1 = [val_akt1,random_val_akt1]

training_pi3k = [train_pi3k,random_train_pi3k]
validation_pi3k = [val_pi3k,random_val_pi3k]

In [14]:
eval_p38 = {}
eval_akt1 = {}
eval_pi3k = {}
es2 = EarlyStopping(monitor='loss',patience=15, min_delta=0)
rlr2 = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=2, verbose=1, min_lr=0.00001)
for i in range(len(training_p38)):
    
        #First Target
        X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38 = dataframe_to_gcn_input(validation_p38[i])
        Y_cold_p38 = validation_p38[i].Binary
        Y_dummy_cold_p38 = np.empty((X_atoms_cold_p38.shape[0],int(model_params['dense_size'][2]/3)+1))
        X_atoms_train_p38, X_bonds_train_p38, X_edges_train_p38 = dataframe_to_gcn_input(training_p38[i])
        Y_p38 = training_p38[i].Binary
        Y_dummy_train_p38 = np.empty((X_atoms_train_p38.shape[0],int(model_params['dense_size'][2]/3)+1))
        smiles_list_p38 = list(training_p38[i]['rdkit'])
        
        #Second Target
        X_atoms_cold_akt1, X_bonds_cold_akt1, X_edges_cold_akt1 = dataframe_to_gcn_input(validation_akt1[i])
        Y_cold_akt1 = validation_akt1[i].Binary
        Y_dummy_cold_akt1 = np.empty((X_atoms_cold_akt1.shape[0],int(model_params['dense_size'][2]/3)+1))
        X_atoms_train_akt1, X_bonds_train_akt1, X_edges_train_akt1 = dataframe_to_gcn_input(training_akt1[i])
        Y_akt1 = training_akt1[i].Binary
        Y_dummy_train_akt1 = np.empty((X_atoms_train_akt1.shape[0],int(model_params['dense_size'][2]/3)+1))
        smiles_list_akt1 = list(training_akt1[i]['rdkit'])
        
        #Third Target
        X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k = dataframe_to_gcn_input(validation_pi3k[i])
        Y_cold_pi3k = validation_pi3k[i].Binary
        Y_dummy_cold_pi3k = np.empty((X_atoms_cold_pi3k.shape[0],int(model_params['dense_size'][2]/3)+1))
        X_atoms_train_pi3k, X_bonds_train_pi3k, X_edges_train_pi3k = dataframe_to_gcn_input(training_pi3k[i])
        Y_pi3k = training_pi3k[i].Binary
        Y_dummy_train_pi3k = np.empty((X_atoms_train_pi3k.shape[0],int(model_params['dense_size'][2]/3)+1))
        smiles_list_pi3k = list(training_pi3k[i]['rdkit'])
        
        gcn_encoder = build_encoder(model_params)
        gcn_model = build_model(model_params,gcn_encoder)
        gcn_mining = build_mining(model_params,gcn_model)
        train_GEN = csv_generator_mining(model_params['batch_size'],training_p38[i],training_akt1[i],training_pi3k[i],
                                  Y_dummy_train_p38,Y_dummy_train_akt1,Y_dummy_train_pi3k,
                                  smiles_list_p38,smiles_list_akt1,smiles_list_pi3k,
                                  Y_p38,Y_akt1,Y_pi3k,
                                  X_atoms_train_p38, X_bonds_train_p38, X_edges_train_p38,
                                  X_atoms_train_akt1, X_bonds_train_akt1, X_edges_train_akt1,
                                  X_atoms_train_pi3k, X_bonds_train_pi3k, X_edges_train_pi3k)
        NUM_TRAIN = np.sum([len(training_p38[i]),len(training_akt1[i]),len(training_pi3k[i])])
        gcn_mining.fit_generator(train_GEN,
                      steps_per_epoch = ceil(3*NUM_TRAIN/model_params['batch_size']),
                      epochs = model_params['n_epochs'],
                      shuffle = True,
                      validation_data = None,
                      callbacks = [es2,rlr2])
        emb_train_1,t1,t2 = gcn_model.predict([X_atoms_train_p38,X_bonds_train_p38,X_edges_train_p38,
                                        X_atoms_train_p38,X_bonds_train_p38,X_edges_train_p38,
                                        X_atoms_train_p38,X_bonds_train_p38,X_edges_train_p38])
        t1,emb_train_2,t2 = gcn_model.predict([X_atoms_train_akt1,X_bonds_train_akt1,X_edges_train_akt1,
                                        X_atoms_train_akt1,X_bonds_train_akt1,X_edges_train_akt1,
                                        X_atoms_train_akt1,X_bonds_train_akt1,X_edges_train_akt1])
        t1,t2,emb_train_3 = gcn_model.predict([X_atoms_train_pi3k,X_bonds_train_pi3k,X_edges_train_pi3k,
                                        X_atoms_train_pi3k,X_bonds_train_pi3k,X_edges_train_pi3k,
                                        X_atoms_train_pi3k,X_bonds_train_pi3k,X_edges_train_pi3k])


        emb_val_1,t1,t2 = gcn_model.predict([X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38,
                                            X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38,
                                            X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38])
        t1,emb_val_2,t2 = gcn_model.predict([X_atoms_cold_akt1,X_bonds_cold_akt1,X_edges_cold_akt1,
                                            X_atoms_cold_akt1,X_bonds_cold_akt1,X_edges_cold_akt1,
                                            X_atoms_cold_akt1,X_bonds_cold_akt1,X_edges_cold_akt1])
        t1,t2,emb_val_3 = gcn_model.predict([X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k,
                                            X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k,
                                            X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k])
        
        del t1,t2, gcn_mining,gcn_encoder,gcn_model,X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38
        del X_atoms_train_p38, X_bonds_train_p38, X_edges_train_p38,X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k
        del X_atoms_train_pi3k, X_bonds_train_pi3k, X_edges_train_pi3k,X_atoms_cold_akt1, X_bonds_cold_akt1, X_edges_cold_akt1
        del X_atoms_train_akt1, X_bonds_train_akt1, X_edges_train_akt1,train_GEN
        
        #First
        dmatrix_train_1 = xgb.DMatrix(data = emb_train_1,label = Y_p38)
        dmatrix_cold_1  = xgb.DMatrix(data = emb_val_1,label = Y_cold_p38)
        evalist_1 = [(dmatrix_train_1,'train'),(dmatrix_cold_1,'eval')]
        xgb_model_1 = xgb.train(xgb_hyper,dmatrix_train_1,300,evalist_1,verbose_eval=True)
        xgb_pred_cold_1 = xgb_model_1.predict(dmatrix_cold_1)
        
        
        #Second
        dmatrix_train_2 = xgb.DMatrix(data = emb_train_2,label = Y_akt1)
        dmatrix_cold_2  = xgb.DMatrix(data = emb_val_2,label = Y_cold_akt1)
        evalist_2 = [(dmatrix_train_2,'train'),(dmatrix_cold_2,'eval')]
        xgb_model_2 = xgb.train(xgb_hyper,dmatrix_train_2,300,evalist_2,verbose_eval=True)
        xgb_pred_cold_2 = xgb_model_2.predict(dmatrix_cold_2)
        
        
        #Third
        dmatrix_train_3 = xgb.DMatrix(data = emb_train_3,label = Y_pi3k)
        dmatrix_cold_3  = xgb.DMatrix(data = emb_val_3,label = Y_cold_pi3k)
        evalist_3 = [(dmatrix_train_3,'train'),(dmatrix_cold_3,'eval')]
        xgb_model_3 = xgb.train(xgb_hyper,dmatrix_train_3,300,evalist_3,verbose_eval=True)
        xgb_pred_cold_3 = xgb_model_3.predict(dmatrix_cold_3)
        
        if i == 0:
            eval_p38['Test'] = calculate_metrics(np.array(Y_cold_p38),xgb_pred_cold_1)
            eval_akt1['Test'] = calculate_metrics(np.array(Y_cold_akt1),xgb_pred_cold_2)
            eval_pi3k['Test'] = calculate_metrics(np.array(Y_cold_pi3k),xgb_pred_cold_3)
        elif i == 1:
            eval_p38['Random'] = calculate_metrics(np.array(Y_cold_p38),xgb_pred_cold_1)
            eval_akt1['Random'] = calculate_metrics(np.array(Y_cold_akt1),xgb_pred_cold_2)
            eval_pi3k['Random'] = calculate_metrics(np.array(Y_cold_pi3k),xgb_pred_cold_3)

LAYER 0
LAYER 1
LAYER 2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
atom_inputs_1 (InputLayer)      (None, 70, 62)       0                                            
__________________________________________________________________________________________________
bond_inputs_1 (InputLayer)      (None, 70, 5, 6)     0                                            
__________________________________________________________________________________________________
edge_inputs_1 (InputLayer)      (None, 70, 5)        0                                            
__________________________________________________________________________________________________
atom_inputs_2 (InputLayer)      (None, 70, 62)       0                                            
_____________________________________________________________________________________

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
95/95 [==============================] - 25s 267ms/step - loss: 1.4795 - Protein_1_loss: 0.4931 - Protein_2_loss: 0.4926 - Protein_3_loss: 0.4938
Epoch 2/20
95/95 [==============================] - 16s 166ms/step - loss: 1.4654 - Protein_1_loss: 0.4927 - Protein_2_loss: 0.4801 - Protein_3_loss: 0.4926
Epoch 3/20
95/95 [==============================] - 16s 166ms/step - loss: 1.4668 - Protein_1_loss: 0.4930 - Protein_2_loss: 0.4812 - Protein_3_loss: 0.4925
Epoch 4/20
95/95 [==============================] - 16s 166ms/step - loss: 1.4642 - Protein_1_loss: 0.4929 - Protein_2_loss: 0.4797 - Protein_3_loss: 0.4916
Epoch 5/20
95/95 [==============================] - 16s 167ms/step - loss: 1.4581 - Protein_1_loss: 0.4933 - Protein_2_loss: 0.4754 - Protein_3_loss: 0.4894
Epoch 6/20
95/95 [==============================] - 16s 167ms/step - loss: 1.4562 - Protein_1_loss: 0.4924 - Protein_2_loss: 0.4741 - Protein_3_loss: 0.4897
Epoch 7/20
95/95 [==============================] - 16s 16

[125]	train-auc:0.84648	eval-auc:0.48954
[126]	train-auc:0.84738	eval-auc:0.48136
[127]	train-auc:0.84807	eval-auc:0.48076
[128]	train-auc:0.85171	eval-auc:0.48411
[129]	train-auc:0.85333	eval-auc:0.48662
[130]	train-auc:0.85091	eval-auc:0.49135
[131]	train-auc:0.85266	eval-auc:0.48968
[132]	train-auc:0.85358	eval-auc:0.49218
[133]	train-auc:0.85141	eval-auc:0.49101
[134]	train-auc:0.85281	eval-auc:0.49132
[135]	train-auc:0.85388	eval-auc:0.48765
[136]	train-auc:0.85333	eval-auc:0.49680
[137]	train-auc:0.85594	eval-auc:0.48762
[138]	train-auc:0.85672	eval-auc:0.48810
[139]	train-auc:0.85967	eval-auc:0.48758
[140]	train-auc:0.85970	eval-auc:0.49134
[141]	train-auc:0.85941	eval-auc:0.49148
[142]	train-auc:0.85710	eval-auc:0.49453
[143]	train-auc:0.85883	eval-auc:0.49646
[144]	train-auc:0.86071	eval-auc:0.49366
[145]	train-auc:0.86145	eval-auc:0.49388
[146]	train-auc:0.86158	eval-auc:0.50000
[147]	train-auc:0.86367	eval-auc:0.50048
[148]	train-auc:0.86382	eval-auc:0.49651
[149]	train-auc:

[26]	train-auc:0.96155	eval-auc:0.86069
[27]	train-auc:0.96176	eval-auc:0.85971
[28]	train-auc:0.96149	eval-auc:0.85985
[29]	train-auc:0.96175	eval-auc:0.86041
[30]	train-auc:0.96191	eval-auc:0.86010
[31]	train-auc:0.96226	eval-auc:0.86019
[32]	train-auc:0.96225	eval-auc:0.86109
[33]	train-auc:0.96242	eval-auc:0.86128
[34]	train-auc:0.96290	eval-auc:0.85987
[35]	train-auc:0.96275	eval-auc:0.86041
[36]	train-auc:0.96283	eval-auc:0.85987
[37]	train-auc:0.96315	eval-auc:0.86016
[38]	train-auc:0.96314	eval-auc:0.86089
[39]	train-auc:0.96316	eval-auc:0.86059
[40]	train-auc:0.96340	eval-auc:0.86064
[41]	train-auc:0.96329	eval-auc:0.86014
[42]	train-auc:0.96341	eval-auc:0.85903
[43]	train-auc:0.96353	eval-auc:0.85871
[44]	train-auc:0.96354	eval-auc:0.85907
[45]	train-auc:0.96342	eval-auc:0.85916
[46]	train-auc:0.96401	eval-auc:0.84764
[47]	train-auc:0.96419	eval-auc:0.84814
[48]	train-auc:0.96415	eval-auc:0.84732
[49]	train-auc:0.96435	eval-auc:0.84714
[50]	train-auc:0.96410	eval-auc:0.84782


...

Epoch 1/20
95/95 [==============================] - 25s 262ms/step - loss: 1.4780 - Protein_1_loss: 0.4926 - Protein_2_loss: 0.4940 - Protein_3_loss: 0.4914
Epoch 2/20
95/95 [==============================] - 16s 166ms/step - loss: 1.4656 - Protein_1_loss: 0.4914 - Protein_2_loss: 0.4849 - Protein_3_loss: 0.4893
Epoch 3/20
95/95 [==============================] - 16s 168ms/step - loss: 1.4579 - Protein_1_loss: 0.4907 - Protein_2_loss: 0.4751 - Protein_3_loss: 0.4921
Epoch 4/20
95/95 [==============================] - 16s 167ms/step - loss: 1.4582 - Protein_1_loss: 0.4904 - Protein_2_loss: 0.4776 - Protein_3_loss: 0.4902
Epoch 5/20
95/95 [==============================] - 16s 167ms/step - loss: 1.4550 - Protein_1_loss: 0.4915 - Protein_2_loss: 0.4734 - Protein_3_loss: 0.4902
Epoch 6/20
95/95 [==============================] - 16s 167ms/step - loss: 1.4547 - Protein_1_loss: 0.4915 - Protein_2_loss: 0.4726 - Protein_3_loss: 0.4905
Epoch 7/20
95/95 [==============================] - 16s 16

[126]	train-auc:0.87291	eval-auc:0.60063
[127]	train-auc:0.87231	eval-auc:0.60011
[128]	train-auc:0.87479	eval-auc:0.59890
[129]	train-auc:0.87523	eval-auc:0.59803
[130]	train-auc:0.87553	eval-auc:0.59405
[131]	train-auc:0.87612	eval-auc:0.59873
[132]	train-auc:0.87470	eval-auc:0.59255
[133]	train-auc:0.87501	eval-auc:0.59034
[134]	train-auc:0.87722	eval-auc:0.59076
[135]	train-auc:0.87639	eval-auc:0.58910
[136]	train-auc:0.87761	eval-auc:0.59152
[137]	train-auc:0.87783	eval-auc:0.59093
[138]	train-auc:0.88008	eval-auc:0.58938
[139]	train-auc:0.88001	eval-auc:0.58999
[140]	train-auc:0.88016	eval-auc:0.59192
[141]	train-auc:0.88247	eval-auc:0.58949
[142]	train-auc:0.88229	eval-auc:0.58487
[143]	train-auc:0.88265	eval-auc:0.58779
[144]	train-auc:0.88339	eval-auc:0.58849
[145]	train-auc:0.88345	eval-auc:0.58935
[146]	train-auc:0.88429	eval-auc:0.58996
[147]	train-auc:0.88536	eval-auc:0.58896
[148]	train-auc:0.88800	eval-auc:0.59197
[149]	train-auc:0.88791	eval-auc:0.59224
[150]	train-auc:

[27]	train-auc:0.96676	eval-auc:0.86392
[28]	train-auc:0.96706	eval-auc:0.86396
[29]	train-auc:0.96762	eval-auc:0.86392
[30]	train-auc:0.96795	eval-auc:0.86344
[31]	train-auc:0.96858	eval-auc:0.86326
[32]	train-auc:0.96899	eval-auc:0.86185
[33]	train-auc:0.96930	eval-auc:0.86197
[34]	train-auc:0.96946	eval-auc:0.86253
[35]	train-auc:0.96960	eval-auc:0.86277
[36]	train-auc:0.96970	eval-auc:0.86247
[37]	train-auc:0.96965	eval-auc:0.86243
[38]	train-auc:0.96976	eval-auc:0.86207
[39]	train-auc:0.96994	eval-auc:0.86095
[40]	train-auc:0.97003	eval-auc:0.86255
[41]	train-auc:0.97038	eval-auc:0.86207
[42]	train-auc:0.97053	eval-auc:0.86163
[43]	train-auc:0.97046	eval-auc:0.86245
[44]	train-auc:0.97103	eval-auc:0.86294
[45]	train-auc:0.97118	eval-auc:0.86306
[46]	train-auc:0.97152	eval-auc:0.86161
[47]	train-auc:0.97142	eval-auc:0.86237
[48]	train-auc:0.97158	eval-auc:0.86233
[49]	train-auc:0.97149	eval-auc:0.86318
[50]	train-auc:0.97151	eval-auc:0.86362
[51]	train-auc:0.97138	eval-auc:0.86370


[229]	train-auc:0.98536	eval-auc:0.85464
[230]	train-auc:0.98505	eval-auc:0.85480
[231]	train-auc:0.98484	eval-auc:0.85504
[232]	train-auc:0.98520	eval-auc:0.85476
[233]	train-auc:0.98539	eval-auc:0.85456
[234]	train-auc:0.98573	eval-auc:0.85444
[235]	train-auc:0.98613	eval-auc:0.85376
[236]	train-auc:0.98575	eval-auc:0.85476
[237]	train-auc:0.98581	eval-auc:0.85380
[238]	train-auc:0.98608	eval-auc:0.85340
[239]	train-auc:0.98580	eval-auc:0.85412
[240]	train-auc:0.98606	eval-auc:0.85392
[241]	train-auc:0.98604	eval-auc:0.85340
[242]	train-auc:0.98617	eval-auc:0.85259
[243]	train-auc:0.98614	eval-auc:0.85328
[244]	train-auc:0.98582	eval-auc:0.85400
[245]	train-auc:0.98592	eval-auc:0.85452
[246]	train-auc:0.98613	eval-auc:0.85408
[247]	train-auc:0.98613	eval-auc:0.85408
[248]	train-auc:0.98620	eval-auc:0.85392
[249]	train-auc:0.98596	eval-auc:0.85476
[250]	train-auc:0.98594	eval-auc:0.85496
[251]	train-auc:0.98614	eval-auc:0.85464
[252]	train-auc:0.98644	eval-auc:0.85359
[253]	train-auc:

[132]	train-auc:0.89863	eval-auc:0.53454
[133]	train-auc:0.89752	eval-auc:0.53551
[134]	train-auc:0.89860	eval-auc:0.53814
[135]	train-auc:0.90029	eval-auc:0.53452
[136]	train-auc:0.90087	eval-auc:0.53213
[137]	train-auc:0.90074	eval-auc:0.53226
[138]	train-auc:0.90149	eval-auc:0.53026
[139]	train-auc:0.90095	eval-auc:0.53199
[140]	train-auc:0.90270	eval-auc:0.53036
[141]	train-auc:0.90188	eval-auc:0.53333
[142]	train-auc:0.90378	eval-auc:0.53520
[143]	train-auc:0.90338	eval-auc:0.53624
[144]	train-auc:0.90550	eval-auc:0.53693
[145]	train-auc:0.90606	eval-auc:0.53768
[146]	train-auc:0.90533	eval-auc:0.53915
[147]	train-auc:0.90540	eval-auc:0.53585
[148]	train-auc:0.90610	eval-auc:0.53632
[149]	train-auc:0.90621	eval-auc:0.53595
[150]	train-auc:0.90737	eval-auc:0.54046
[151]	train-auc:0.90764	eval-auc:0.54064
[152]	train-auc:0.90869	eval-auc:0.53939
[153]	train-auc:0.90769	eval-auc:0.53744
[154]	train-auc:0.90701	eval-auc:0.53769
[155]	train-auc:0.91008	eval-auc:0.53709
[156]	train-auc:

In [18]:
eval_p38 = pd.DataFrame(eval_p38).T
eval_p38.to_csv('../../../../Desktop/binding/thesis english/Results/4-Multitask/Online/p38.csv')

eval_akt1 = pd.DataFrame(eval_akt1).T
eval_akt1.to_csv('../../../../Desktop/binding/thesis english/Results/4-Multitask/Online/akt1.csv')

eval_pi3k = pd.DataFrame(eval_pi3k).T
eval_pi3k.to_csv('../../../../Desktop/binding/thesis english/Results/4-Multitask/Online/pi3k.csv')

In [20]:
eval_p38

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Test,0.503775,161.0,145.0,102.0,101.0,0.416844,0.410569,0.497537,0.514735
Random,0.588056,182.0,118.0,109.0,125.0,0.522963,0.514403,0.534188,0.574906


In [21]:
eval_akt1

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Test,0.848480,170.0,20.0,47.0,69.0,0.786818,0.775281,0.594828,0.781046
Random,0.856802,158.0,31.0,35.0,97.0,0.839379,0.757812,0.734848,0.794393


In [22]:
eval_pi3k

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Test,0.641557,293.0,67.0,112.0,65.0,0.466434,0.492424,0.367232,0.666667
Random,0.534291,270.0,93.0,149.0,52.0,0.372734,0.358621,0.258706,0.570922
